# time_lapse
Author: Connor Finn <br>
Date: July 26, 2020 <br>

Description: <dir>
    This is a script which generates a time lapse vido. For this, it is recomended that the 'LANDSAT/LC08/C01/T1_TOA' satelite is used. 

# Set Up

In [1]:
import ee
import numpy as np
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/2gGaiByIcXSamQsu_i1l0pOlPbFTWXIGLs-9EnQW2v1n_uYYY4xvkN4

Successfully saved authorization token.


In [2]:
ee.Initialize()

# Time Lapse Class

In [3]:
class Time_Lapse_Builder():
    
    
    def __init__(self):
        self.collection = None
        

    def get_collection(self, path_list , row_list , satelite , start_date , end_date , max_cloud_percentage , months):

        # get image collection object
        coll = ee.ImageCollection(satelite)\
            .filterDate(start_date, end_date)\
            .filter(ee.Filter.inList('WRS_PATH', path_list))\
            .filter(ee.Filter.inList('WRS_ROW', row_list))\
            .filter(ee.Filter.lt('CLOUD_COVER' , max_cloud_percentage))\
            .filter(ee.Filter.calendarRange(months[0],months[1],'month'))    # just may data
        self.collection = coll
        
        
    def build_movie(self , region , band_list , file_name):
        from ee import batch
        # select the bands we want to video
        bands = self.collection.select(band_list)

        # call the conversion    
        outputVideo = bands.map(lambda x: x.multiply(512).uint8())

        #Export to video.
        out = batch.Export.video.toDrive( outputVideo, description= file_name, dimensions = 720,\
                            framesPerSecond = 2, region= region, maxFrames=10000)

        # process the video
        process = batch.Task.start(out)
        
    def build_movie_url(self , region , band_list):
        bands = self.collection.select(band_list)
        outputVideo = bands.map(lambda x: x.multiply(512).uint8())
        vid = outputVideo.getVideoThumbURL({
              'dimensions': 720,
              'bands': band_list, 
              'framesPerSecond' : 2,
               'region' : ee.Geometry.Polygon([region]) 
        })
    
        return vid
        
    

# use the class

## Set up

In [8]:
# initialize the class
mc = Time_Lapse_Builder()

# fill in the information
p = [39]                                    # path       
r = [37]                                    # row 
sat = 'LANDSAT/LC08/C01/T1_TOA'             # use this satelite
sd = '2013-05-01'                           # start date
ed = '2020-05-01'                           # end date
cc= 20                                      # max cloud cover percentage
months = [5 , 5]                            # months [start, end]

# collect the images
mc.get_collection(p, r, sat, sd, ed, cc , months)

# this surrounds the box
lake_bounding_box = ([-116.2170533393782,33.578034161036506],\
 [-116.1895875190657,33.03171253668441],\
 [-115.3601197456282,33.045527348747925],\
 [-115.4040650581282,33.64665492287974])

# rgb bands
band_list = ['B4', 'B3', 'B2']

## Send movie to Drive
The code in this section will send your video to your current google drive account

In [9]:
file_name = 'salton_sea_TimeLapse'
mc.build_movie(lake_bounding_box, band_list , file_name)

## Display movie Via Url
I couldn't get this to show up in the notebook using ipython.display. for now just paste the resultant url in your drive

In [10]:
mc.build_movie_url(lake_bounding_box, band_list)

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/ad85b98bc7e7b555759b098dbe38325b-9ba13ccbb30845f71edf467de8f7b282:getPixels'

# california path row

This is a map of the landsat path and rows for california

In [7]:
Image(url = "http://www.edc.uri.edu/nrs/classes/nrs409509/RS/Lectures/409509SatelliteRemoteSensing-2014.pdf")